# Import temperature data from the DWD and process it

This notebook pulls historical temperature data from the DWD server and formats it for future use in other projects. The data is reported hourly for each of the available weather stations and packaged in a zip file. To use the data, we need to store everything in a single .csv file, all stations side-by-side. Also, we need the daily average.

To reduce computing time, we also exclude data earlier than 2007.

Files should be executed in the following pipeline:
* 1-dwd_konverter_download
* 2-dwd_konverter_extract
* 3-dwd_konverter_build_df
* 4-dwd_konverter_final_processing

## 4.) Final data processing
We load in the data that has been saved in the last step, so we don't need to repeat our calculations if we pause  and come back later. 
### Data Cleaning
The data contains some errors, which need to be cleaned. Looking at the output of main_df.describe() in the last cell, you can see that the minimum teperature on some stations is -999. That leaves us with no plausible measurement for that particular hour. Changing this to np.nan allows us to safely calculate the avarage values. 
### Change the frequency
Finally we resample the data to daily means.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

# Import and export paths
pkl_file = Path.cwd() / "export_uncleaned" / "to_clean.pkl"
cleaned_file = Path.cwd() / "export_cleaned" / "cleaned.csv"

# Read in the pickle file from the last cell
cleaning_df = pd.read_pickle(pkl_file)


# Replace all values with "-999", which indicate missing data
cleaning_df.replace(to_replace=-999, value=np.nan, inplace=True)

# Resample to daily frequency
cleaning_df = cleaning_df.resample('D').mean().round(decimals=2)

# Save as .csv
cleaning_df.to_csv(cleaned_file, sep=";", decimal=",")

display(cleaning_df.loc['2011-12-31':'2012-01-04'])
display(cleaning_df.describe())
display(cleaning_df)

TT_TU                                                 
STATIONS_ID   3      44    71    73     78     91  96    102   125
MESS_DATUM                                                        
2011-12-31    NaN   3.88  2.76  1.19   4.30   2.43 NaN  3.80   NaN
2012-01-01    NaN  10.90  8.14  4.03  10.96  10.27 NaN  9.01   NaN
2012-01-02    NaN   7.41  6.18  4.77   7.57   7.77 NaN  6.48  4.66
2012-01-03    NaN   6.14  3.61  4.46   6.38   5.28 NaN  5.63  3.51
2012-01-04    NaN   5.80  2.48  4.45   5.46   4.57 NaN  5.85  1.94

TT_TU                                                      \
STATIONS_ID          3            44           71           73           78    
count        1551.000000  4995.000000  3683.000000  5018.000000  5114.000000   
mean           10.103939    10.147512     8.411244     9.705891     9.944949   
std             6.742460     6.588813     7.511708     7.795431     6.599975   
min           -10.870000   -10.710000   -14.940000   -14.320000   -12.390000   
25%             5.410000     5.375000     2.620000     3.412500     5.180000   
50%            10.140000    10.350000     8.570000     9.950000     9.955000   
75%            15.350000    15.330000    14.070000    16.047500    15.120000   
max            28.410000    28.450000    27.190000    27.030000    29.890000   

                                                                
STATIONS_ID          91          96           102          125  
count        5112.000000  633.000000  4856.000000  4301.000000  
mean            9.265892   11.777093    10.283785     8.492823  
std             7.073431    6.704972     6.021094     7.639642  
min           -15.710000   -1.160000    -8.170000   -16.420000  
25%             3.920000    5.920000     5.867500     2.450000  
50%             9.315000   11.720000    10.210000     8.590000  
75%            14.840000   16.880000    15.292500    14.520000  
max            27.550000   27.800000    27.330000    28.030000

TT_TU                                              
STATIONS_ID   3     44  71    73    78    91    96    102   125
MESS_DATUM                                                     
2007-01-01   7.38   NaN NaN   NaN  7.42  6.55   NaN  8.32   NaN
2007-01-02   4.67   NaN NaN   NaN  4.49  2.88   NaN  6.73  0.51
2007-01-03   6.19   NaN NaN   NaN  4.87  4.25   NaN  7.12  0.91
2007-01-04   7.69   NaN NaN   NaN  7.82  5.85   NaN  8.34  4.43
2007-01-05   7.78   NaN NaN   NaN  7.47  6.03   NaN  8.20  3.92
...           ...   ...  ..   ...   ...   ...   ...   ...   ...
2020-12-27    NaN  3.89 NaN -2.72  4.16  0.78  1.54  4.47 -5.86
2020-12-28    NaN  3.10 NaN  0.79  3.02  2.13  2.09  3.78 -1.40
2020-12-29    NaN  2.93 NaN  0.04  2.78  1.59  1.48  2.95 -1.63
2020-12-30    NaN  3.61 NaN -0.45  3.16  1.92  1.71  4.38 -1.20
2020-12-31    NaN  2.50 NaN -0.15  2.03  1.44  1.74  3.56 -2.54

[5114 rows x 9 columns]